In [1]:
#Submission to Kaggle Titanic Competition
import pandas as pd
DF = pd.read_csv('train.csv')
test_DF = pd.read_csv('test.csv')
DF.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

# Splitting
from sklearn.model_selection import train_test_split

# Models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



# Metrics
from sklearn.metrics import accuracy_score, f1_score

# Grid Search
from sklearn.model_selection import GridSearchCV

In [3]:
DF.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [4]:
DF.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [5]:
DF.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
PASSENGER_ID = test_DF[['PassengerId']]
DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)
test_DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)

In [7]:

labels = test_DF.columns
for col in labels:
    if col in ['Age', 'RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
        DF[col].fillna(DF[col].median(), inplace=True)
        test_DF[col].fillna(DF[col].median(), inplace=True)
    else:
        DF[col].fillna(DF[col].mode()[0], inplace=True)
        test_DF[col].fillna(DF[col].mode()[0], inplace=True)
for col in labels:
    if DF[col].dtype == 'O':
        encoder = LabelEncoder()
        DF[col] = encoder.fit_transform(DF[col])
        test_DF[col] = encoder.transform(test_DF[col])
        
    elif DF[col].dtype == 'bool':
        DF[col] = DF[col].astype('int')
        test_DF[col] = test_DF[col].astype('int')
encoder = LabelEncoder()
DF['Transported'] = DF['Transported'].astype('int')
labels_MM = ['Age']
labels_SS = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [8]:
mm_scaler = MinMaxScaler()
ss_scaler = StandardScaler()

DF[labels_MM] = mm_scaler.fit_transform(DF[labels_MM])
test_DF[labels_MM] = mm_scaler.transform(test_DF[labels_MM])


DF[labels_SS] = ss_scaler.fit_transform(DF[labels_SS])
test_DF[labels_SS] = ss_scaler.transform(test_DF[labels_SS])



In [9]:
X, y =DF.drop('Transported', axis=1), np.array(DF[['Transported']]).reshape(-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, stratify=y)

y

array([0, 1, 0, ..., 1, 0, 1])

In [10]:
model_accuracy = {}

In [11]:
ensemble_models = GaussianNB()
predictor_gnb = ensemble_models.fit(X_train, y_train)
y_pred_gnb = predictor_gnb.predict(X_test)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
model_accuracy['naive_bayes'] = accuracy_gnb
print("Accuracy of Gaussian Naive Bayes :",accuracy_gnb)

Accuracy of Gaussian Naive Bayes : 0.7057471264367816


In [12]:
ensemble_models = LogisticRegression()
predictor_lr = ensemble_models.fit(X_train, y_train)
y_pred_lr = predictor_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
model_accuracy['logistic_regression'] = accuracy_lr
print("Accuracy of Logistic Regression :",accuracy_lr)

Accuracy of Logistic Regression : 0.7701149425287356


In [13]:
ensemble_models = SVC()
predictor_svc = ensemble_models.fit(X_train, y_train)
y_pred_svc = predictor_svc.predict(X_test)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
model_accuracy['SVC'] = accuracy_svc
print("Accuracy of SVC :",accuracy_svc)

Accuracy of SVC : 0.774712643678161


In [14]:
ensemble_models = DecisionTreeClassifier()
predictor_dtc = ensemble_models.fit(X_train, y_train)
y_pred_dtc = predictor_dtc.predict(X_test)
accuracy_dtc = accuracy_score(y_test, y_pred_dtc)
model_accuracy['decision_tree'] = accuracy_dtc
print("Accuracy of Decision Tree :",accuracy_dtc)


Accuracy of Decision Tree : 0.7379310344827587


In [15]:
ensemble_models = RandomForestClassifier()
predictor_rtc = ensemble_models.fit(X_train, y_train)
y_pred_rtc = predictor_rtc.predict(X_test)
accuracy_rtc = accuracy_score(y_test, y_pred_rtc)
model_accuracy['random_forest'] = accuracy_rtc
print("Accuracy of Random Tree :",accuracy_rtc)

Accuracy of Random Tree : 0.7885057471264367


In [16]:
ensemble_models = GradientBoostingClassifier()
predictor_gb = ensemble_models.fit(X_train, y_train)
y_pred_gb = predictor_gb.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
model_accuracy['gradient_boost'] = accuracy_gb
print("Accuracy of Gradient Boost :",accuracy_gb)

Accuracy of Gradient Boost : 0.7954022988505747


In [17]:
ensemble_models = MLPClassifier()
predictor_mlp = ensemble_models.fit(X_train, y_train)
y_pred_mlp = predictor_mlp.predict(X_test)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
model_accuracy['MLP'] = accuracy_mlp
print("Accuracy of MLP :",accuracy_mlp)

Accuracy of MLP : 0.7965517241379311


In [18]:
ensemble_models = LinearDiscriminantAnalysis()
predictor_lda = ensemble_models.fit(X_train, y_train)
y_pred_lda = predictor_lda.predict(X_test)
accuracy_lda = accuracy_score(y_test, y_pred_lda)
model_accuracy['LDA'] = accuracy_lda
print("Accuracy of LDA :",accuracy_lda)

Accuracy of LDA : 0.7390804597701149


In [19]:
ensemble_models = KNeighborsClassifier()
predictor_knn = ensemble_models.fit(X_train, y_train)
y_pred_knn = predictor_knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
model_accuracy['KNN'] = accuracy_knn
print("Accuracy of KNN :",accuracy_knn)

Accuracy of KNN : 0.774712643678161


c:\Users\lihao\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [20]:
ensemble_models = AdaBoostClassifier()
predictor_ada = ensemble_models.fit(X_train, y_train)
y_pred_ada = predictor_ada.predict(X_test)
accuracy_ada = accuracy_score(y_test, y_pred_ada)
model_accuracy['ADA'] = accuracy_ada
print("Accuracy of ADA :",accuracy_ada)

Accuracy of ADA : 0.7862068965517242


In [21]:
model_accuracy

{'naive_bayes': 0.7057471264367816,
 'logistic_regression': 0.7701149425287356,
 'SVC': 0.774712643678161,
 'decision_tree': 0.7379310344827587,
 'random_forest': 0.7885057471264367,
 'gradient_boost': 0.7954022988505747,
 'MLP': 0.7965517241379311,
 'LDA': 0.7390804597701149,
 'KNN': 0.774712643678161,
 'ADA': 0.7862068965517242}

In [22]:
voting_ensemble = VotingClassifier(estimators= [('GB', predictor_gb), ('MLP', predictor_mlp), ('SVC', predictor_svc), ('GNB',predictor_gnb), ('LR',predictor_lr), ('DT', predictor_dtc), ('RT', predictor_rtc), ('LDA', predictor_lda), ('KNN', predictor_knn), ('ADA', predictor_ada)],voting = 'hard')

ensemble_predictor = voting_ensemble.fit(X_train, y_train)

y_pred = ensemble_predictor.predict(X_test)
print(f'Accuracy scores for the first ensemble: {accuracy_score(y_test, y_pred)}')

c:\Users\lihao\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy scores for the first ensemble: 0.7862068965517242


c:\Users\lihao\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [23]:
ensemble_predictor

VotingClassifier(estimators=[('GB', GradientBoostingClassifier()),
                             ('MLP', MLPClassifier()), ('SVC', SVC()),
                             ('GNB', GaussianNB()),
                             ('LR', LogisticRegression()),
                             ('DT', DecisionTreeClassifier()),
                             ('RT', RandomForestClassifier()),
                             ('LDA', LinearDiscriminantAnalysis()),
                             ('KNN', KNeighborsClassifier()),
                             ('ADA', AdaBoostClassifier())])

In [24]:
submission = ensemble_predictor.predict(test_DF)

data = {'PassengerId': np.array(PASSENGER_ID).reshape(len(PASSENGER_ID)),
        'Transported': np.array(submission).astype('bool')}

submission_df = pd.DataFrame(data=data).reset_index()
submission_df.drop(columns=['index'], inplace=True, axis=1)
# submission_df.to_csv("submission.csv", index=False)


c:\Users\lihao\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [25]:
#Class Submission
from pandas import read_csv
from sklearn.model_selection import train_test_split
_DF = read_csv('train_.csv')
_test_DF = read_csv('test_.csv')
_validation_DF = read_csv('validation_.csv')


In [26]:
_DF.dtypes

Unnamed: 0        int64
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [27]:
_DF.isnull().sum()

Unnamed: 0        0
PassengerId       0
HomePlanet      142
CryoSleep       156
Cabin           144
Destination     122
Age             130
VIP             152
RoomService     126
FoodCourt       126
ShoppingMall    151
Spa             118
VRDeck          129
Name            145
Transported       0
dtype: int64

In [28]:
PASSENGER_ID = _test_DF[['PassengerId']]
_DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)
_test_DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)
_validation_DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)

In [29]:
from sklearn.preprocessing import LabelEncoder
labels = _test_DF.columns
for col in labels:
    if col in ['Age', 'RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
        _DF[col].fillna(_DF[col].median(), inplace=True)
        _test_DF[col].fillna(_DF[col].median(), inplace=True)
        _validation_DF[col].fillna(_DF[col].median(), inplace=True)
    else:
        _DF[col].fillna(_DF[col].mode()[0], inplace=True)
        _test_DF[col].fillna(_DF[col].mode()[0], inplace=True)
        _validation_DF[col].fillna(_DF[col].mode()[0], inplace=True)
for col in labels:
    if _DF[col].dtype == 'O':
        encoder = LabelEncoder()
        _DF[col] = encoder.fit_transform(_DF[col])
        _test_DF[col] = encoder.transform(_test_DF[col])
        _validation_DF[col] = encoder.transform(_validation_DF[col])
        
    elif _DF[col].dtype == 'bool':
        _DF[col] = _DF[col].astype('int')
        _test_DF[col] = _test_DF[col].astype('int')
        _validation_DF[col] = _validation_DF[col].astype('int')
encoder = LabelEncoder()
_DF['Transported'] = _DF['Transported'].astype('int')
labels_MM = ['Age']
labels_SS = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [30]:
print(_DF.dtypes)
print(_DF.isnull().sum())
print(_test_DF.isnull().sum())
print(_validation_DF.isnull().sum())

Unnamed: 0        int64
HomePlanet        int32
CryoSleep         int32
Destination       int32
Age             float64
VIP               int32
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Transported       int32
dtype: object
Unnamed: 0      0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64
Unnamed: 0      0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64
Unnamed: 0      0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64


In [31]:
# _X_train, _X_test, _y_train, _y_test = train_test_split(_DF.drop(['Transported'], axis=1), _DF['Transported'], test_size=0.2, random_state=42)
_X_train = _DF.drop(['Transported'], axis=1)
_y_train = _DF['Transported']
_X_test = _validation_DF.drop(['Transported'], axis=1)
_y_test = _validation_DF['Transported']
_X_final_test = _test_DF.drop(['Transported'], axis=1)
_y_final_test = _test_DF['Transported']

In [32]:
#Olof Persson's code

In [33]:
#Parametric model
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
#create a naive bayes classifier with grid search parameters for tuning

model = GaussianNB()
# define grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict()
space['var_smoothing'] = [1e-05,1e-06,1e-07,1e-08,1e-09, 1e-10, 1e-11, 1e-12, 1e-13]
# define search
search = GridSearchCV(model, space, scoring='accuracy', n_jobs=-1, cv=cv)
# execute search
result = search.fit(_X_train, _y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.6979888391906283
Best Hyperparameters: {'var_smoothing': 1e-08}


In [34]:
#Non-parametric model
#Neural Network
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
#create a neural network classifier with grid search parameters for tuning

model = MLPClassifier()
# define grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict()
space['hidden_layer_sizes'] = [(10,30,10),(20,),(50,),(100,)]
space['activation'] = ['tanh', 'relu']
# space['solver'] = ['lbfgs', 'sgd', 'adam']
# space['alpha'] = [0.0001, 0.05]
space['learning_rate'] = ['constant','adaptive']
# define search
search = GridSearchCV(model, space, scoring='accuracy', n_jobs=-1, cv=cv)
# execute search
result = search.fit(_X_train, _y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.783211501597444
Best Hyperparameters: {'activation': 'tanh', 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive'}


In [35]:
final_model_GNB = GaussianNB(var_smoothing=1e-08)
final_model_MLP = MLPClassifier(activation='tanh', hidden_layer_sizes=(50,), learning_rate='adaptive')
final_model_GNB.fit(_X_final_test, _y_final_test)
final_model_MLP.fit(_X_final_test, _y_final_test)
print('Final Model on Test Data for Gaussian Naive Bayes:',final_model_GNB.score(_X_final_test, _y_final_test))
print('Final Model on Test Data for MLP Classifier Neural Network:',final_model_MLP.score(_X_final_test, _y_final_test))


Final Model on Test Data for Gaussian Naive Bayes: 0.7090281771132835
Final Model on Test Data for MLP Classifier Neural Network: 0.8067855089131685


In [36]:
#Koushik Reddy's code

In [37]:
#Parametric model
#Linear Discriminant Analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# create an LDA classifier with grid search parameters for tuning
model = LinearDiscriminantAnalysis()

# define grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# define search space
space = dict()
space['solver'] = ['svd', 'lsqr', 'eigen']

# define search
search = GridSearchCV(model, space, scoring='accuracy', n_jobs=-1, cv=cv)

# execute search
result = search.fit(_X_train, _y_train)

# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.7562597657082003
Best Hyperparameters: {'solver': 'svd'}


In [38]:
# Train and evaluate final model on test data for LDA
final_model_LDA = LinearDiscriminantAnalysis(solver='svd')
final_model_LDA.fit(_X_final_test, _y_final_test)
print('Final Model on Test Data for Linear Discriminant Analysis:', final_model_LDA.score(_X_test, _y_test))

Final Model on Test Data for Linear Discriminant Analysis: 0.7658045977011494


In [39]:
# Train and evaluate final model on test data for LDA
final_model_LDA = LinearDiscriminantAnalysis(solver='svd')
final_model_LDA.fit(_X_final_test, _y_final_test)
print('Final Model on Test Data for Linear Discriminant Analysis:', final_model_LDA.score(_X_final_test, _y_final_test))

Final Model on Test Data for Linear Discriminant Analysis: 0.7705577918343876


In [40]:
#Non-parametric model
# Support Vector Machines (SVM)
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

# create an SVM classifier with grid search parameters for tuning
model = SVC()

# define grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# define search space
space = dict()
space['C'] = [10, 100]
space['kernel'] = ['poly', 'rbf', 'sigmoid']

# define search
search = GridSearchCV(model, space, scoring='accuracy', n_jobs=-1, cv=cv)

# execute search
result = search.fit(_X_train, _y_train)

# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)


Best Score: 0.7822517571884984
Best Hyperparameters: {'C': 100, 'kernel': 'rbf'}


In [41]:
# Train and evaluate final model on test data for SVC
final_model_SVC = SVC(C=100, kernel='rbf', gamma='scale', degree=3)
final_model_SVC.fit(_X_final_test, _y_final_test)
print('Final Model on Test Data for SVC:', final_model_SVC.score(_X_final_test, _y_final_test))


Final Model on Test Data for SVC: 0.8148361127084531


In [42]:
#Hao Dian's Code

In [43]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model_lr = LogisticRegression()

param_grid_lr = {'C': [0.01, 0.1, 1, 10]}

grid_lr = GridSearchCV(model_lr, param_grid_lr, cv=5)

grid_lr.fit(X_train, y_train)

print("Logistic Regression Best Score:", grid_lr.best_score_)
print("Logistic Regression Best Parameters:", grid_lr.best_params_)

y_pred_lr = grid_lr.predict(X_test)

accuracy_lr = accuracy_score(y_test, y_pred_lr)

print("Logistic Regression Accuracy:", accuracy_lr)

Logistic Regression Best Score: 0.784095503460448
Logistic Regression Best Parameters: {'C': 1}
Logistic Regression Accuracy: 0.7701149425287356


In [44]:
# decision tree
from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier()

param_grid_dt = {'max_depth': [3, 5, 7, 9]}

grid_dt = GridSearchCV(model_dt, param_grid_dt, cv=5)

grid_dt.fit(X_train, y_train)

print("Decision Tree Best Score:", grid_dt.best_score_)
print("Decision Tree Best Parameters:", grid_dt.best_params_)

y_pred_dt = grid_dt.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_dt)

print("Decision Tree Accuracy:", accuracy_dt)

Decision Tree Best Score: 0.7728477811460742
Decision Tree Best Parameters: {'max_depth': 7}
Decision Tree Accuracy: 0.7804597701149425


In [46]:
#Final Model on Test Data for Stacking Classifier
from sklearn.ensemble import StackingClassifier
estimators = [
('lr', LogisticRegression(C=1)),
('dt', DecisionTreeClassifier(max_depth=7)),
('mlp', MLPClassifier(activation='tanh', hidden_layer_sizes=(50,), learning_rate='adaptive')),
('gnb', GaussianNB(var_smoothing=1e-08)),
('svc', SVC(C=100, kernel='rbf', gamma='scale', degree=3))]
final_model_Stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(C=0.1))
final_model_Stacking.fit(_X_final_test, _y_final_test)
print('Final Model on Test Data for Stacking Classifier:', final_model_Stacking.score(_X_final_test, _y_final_test))

c:\Users\lihao\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\lihao\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Final Model on Test Data for Stacking Classifier: 0.8108108108108109
